<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_1_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1055      0 --:--:-- --:--:-- --:--:--  1055
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [10]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [11]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [20]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.846150,4.185192,0.220387,03:44


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.978089,3.704186,0.295796,03:43


In [22]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.708912,3.671263,0.301935,03:42


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.687976,3.649195,0.304725,03:43


In [0]:
learn.save_encoder('ft_fold1_first')

In [25]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.867013,3.731402,0.354722,01:11


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.821135,3.134569,0.466102,01:27


In [27]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.234002,2.776997,0.537530,03:35


In [28]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.061808,2.470470,0.575061,03:38
1,1.794250,2.309678,0.583535,03:39


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.734436,2.130068,0.610169,03:51
1,1.444483,2.022423,0.642857,03:41


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.360616,1.833660,0.671913,03:52
1,1.198675,1.798841,0.675545,03:44


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.166504,1.785076,0.676755,03:37
1,1.040780,1.683557,0.690073,03:49


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.995190,1.654542,0.698547,03:34
1,0.946744,1.581410,0.703390,03:54


In [33]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.866299,1.540249,0.714286,03:42
1,0.867456,1.519962,0.716707,03:44
2,0.762731,1.474512,0.725182,03:54
3,0.702077,1.479334,0.730024,03:43


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.707743,1.469439,0.739709,03:56
1,0.747356,1.444411,0.733656,03:49
2,0.698420,1.436827,0.753027,03:42
3,0.589261,1.393761,0.748184,03:53


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.628026,1.427584,0.743341,03:51
1,0.651428,1.404874,0.746973,03:38
2,0.589713,1.402724,0.769976,03:49
3,0.554861,1.385820,0.765133,03:37


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.571358,1.435774,0.762712,03:39
1,0.590345,1.428293,0.766344,03:26
2,0.551149,1.401517,0.774818,03:46
3,0.511936,1.420513,0.778450,03:32


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.521896,1.424029,0.766344,03:44
1,0.555182,1.411293,0.767554,03:33
2,0.524581,1.418861,0.782082,03:35
3,0.449077,1.362086,0.777240,03:43


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.495887,1.449291,0.771186,03:40
1,0.548164,1.391320,0.769976,03:50
2,0.515497,1.439522,0.778450,03:37
3,0.451958,1.433208,0.773608,03:47


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.490867,1.420133,0.780872,03:35
1,0.495572,1.410321,0.778450,03:29
2,0.484579,1.375974,0.783293,03:24
3,0.463140,1.363152,0.786925,03:34


In [0]:
# save the best model

learn.save_encoder('ask-1-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [42]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[2, 2, 8, 122, 8, 8, 611, 16, 20, 21, 611, 383, 27, 31, 994, 37, 37, 37, 37, 37, 185, 37, 289, 289, 37, 864, 427, 42, 44, 603, 49, 49, 49, 49, 49, 49, 49, 49, 53, 53, 53, 188, 60, 61, 62, 1002, 70, 776, 71, 122, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 260, 289, 289, 289, 289, 289, 289, 289, 260, 80, 1009, 80, 80, 80, 80, 491, 260, 88, 89, 611, 92, 92, 92, 94, 94, 94, 94, 94, 736, 453, 453, 453, 94, 96, 96, 366, 591, 100, 100, 100, 100, 337, 100, 100, 572, 337, 100, 100, 100, 103, 537, 105, 107, 107, 660, 660, 111, 111, 462, 111, 111, 111, 111, 111, 289, 111, 114, 117, 447, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 375, 289, 624, 55, 135, 135, 453, 139, 139, 913, 447, 828, 864, 146, 158, 166, 166, 350, 537, 171, 172, 289, 260, 172, 172, 214, 214, 214, 106, 387, 182, 127, 49, 182, 182, 182, 182, 182, 289, 182, 289, 758, 758, 758, 758, 758, 758, 758, 184,

In [43]:
correct_num = 0

for i in range(0, 867):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

549
0.6332179930795848
